# 🗑️ Delete 5 Records

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import requests

In [2]:
# Create Spark Session
spark = SparkSession.builder \
    .appName("Delete5Records") \
    .master("local[*]") \
    .config("spark.es.nodes", "elasticsearch") \
    .config("spark.es.port", "9200") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:8.14.3") \
    .getOrCreate()

print(f"Application ID: {spark.sparkContext.applicationId}")

Application ID: local-1757837674503


In [3]:
def delete_record_by_id(doc_id):
    """Delete a record from Elasticsearch by ID using HTTP DELETE"""
    url = f"http://elasticsearch:9200/2_people_data_2k/_doc/{doc_id}"
    try:
        response = requests.delete(url)
        return response.status_code == 200
    except Exception as e:
        print(f"Error deleting record {doc_id}: {e}")
        return False

In [4]:
# IDs to delete
delete_ids = [24410114, 24410100, 24410109, 24410092, 24410040]
print(f"Deleting records with IDs: {delete_ids}")

# Read current data to show records before deletion
current_df = spark.read \
    .format("org.elasticsearch.spark.sql") \
    .option("es.nodes", "elasticsearch") \
    .option("es.port", "9200") \
    .option("es.resource", "2_people_data_2k") \
    .load()

# Show records before deletion
records_to_delete = current_df.filter(col("id").isin(delete_ids))
print(f"Records to delete ({records_to_delete.count()}):")
records_to_delete.show()

Deleting records with IDs: [24410114, 24410100, 24410109, 24410092, 24410040]
Records to delete (5):
+----------+--------+---+-----+--------+----+--------+----+-------+--------------------+
|@timestamp|@version|age|event|filename|host|      id| log|message|                name|
+----------+--------+---+-----+--------+----+--------+----+-------+--------------------+
|      NULL|    NULL| 30| NULL|    NULL|NULL|24410100|NULL|   NULL|Nguyen Phuong Tan...|
|      NULL|    NULL| 35| NULL|    NULL|NULL|24410092|NULL|   NULL|Huynh Duy Quoc up...|
|      NULL|    NULL| 22| NULL|    NULL|NULL|24410040|NULL|   NULL| Ha Huy Hung updated|
|      NULL|    NULL| 30| NULL|    NULL|NULL|24410114|NULL|   NULL|Tran Tireu Thuan ...|
|      NULL|    NULL| 28| NULL|    NULL|NULL|24410109|NULL|   NULL|Nguyen Thi Thu Th...|
+----------+--------+---+-----+--------+----+--------+----+-------+--------------------+



In [5]:
# Delete records using HTTP DELETE
print("Deleting records...")
deleted_count = 0
for doc_id in delete_ids:
    if delete_record_by_id(doc_id):
        print(f"✓ Deleted record ID: {doc_id}")
        deleted_count += 1
    else:
        print(f"✗ Failed to delete record ID: {doc_id}")

print(f"Deleted {deleted_count} out of {len(delete_ids)} records")

Deleting records...
✓ Deleted record ID: 24410114
✓ Deleted record ID: 24410100
✓ Deleted record ID: 24410109
✓ Deleted record ID: 24410092
✓ Deleted record ID: 24410040
Deleted 5 out of 5 records


In [6]:
# Verify deletion
print("Verifying deletion...")
updated_df = spark.read \
    .format("org.elasticsearch.spark.sql") \
    .option("es.nodes", "elasticsearch") \
    .option("es.port", "9200") \
    .option("es.resource", "2_people_data_2k") \
    .load()

remaining_records = updated_df.filter(col("id").isin(delete_ids))
remaining_count = remaining_records.count()
total_count = updated_df.count()

print(f"Remaining records with deleted IDs: {remaining_count}")
print(f"Total records after deletion: {total_count}")

if remaining_count > 0:
    print("Remaining records:")
    remaining_records.show()

Verifying deletion...
Remaining records with deleted IDs: 0
Total records after deletion: 2001


In [7]:
# Stop Spark session
spark.stop()
print("Spark session stopped.")

Spark session stopped.
